In [1]:
import torch

print("torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No CUDA device available")
print("CUDA device count:", torch.cuda.device_count())
print("Current CUDA device:", torch.cuda.current_device() if torch.cuda.is_available() else "No current CUDA device")
print("CUDA device capability:", torch.cuda.get_device_capability(0) if torch.cuda.is_available() else "No device capability available")
print("CUDA device properties:", torch.cuda.get_device_properties(0) if torch.cuda.is_available() else "No device properties available")
print("BF16 support:", torch.cuda.is_bf16_supported() if torch.cuda.is_available() else "BF16 support not available")
print("Architecture list:", torch.cuda.get_arch_list() if torch.cuda.is_available() else "No architecture list available")


torch version: 2.7.1+cu118
CUDA available: True
CUDA device name: NVIDIA GeForce RTX 3070 Ti Laptop GPU
CUDA device count: 1
Current CUDA device: 0
CUDA device capability: (8, 6)
CUDA device properties: _CudaDeviceProperties(name='NVIDIA GeForce RTX 3070 Ti Laptop GPU', major=8, minor=6, total_memory=8191MB, multi_processor_count=46, uuid=fefe497f-e5be-51e3-ecb6-db968d4da032, L2_cache_size=4MB)
BF16 support: True
Architecture list: ['sm_50', 'sm_60', 'sm_70', 'sm_75', 'sm_80', 'sm_86', 'sm_37', 'sm_90']


# Check CUDNN

## simple way system find

In [ ]:
!find /usr -name 'libcudnn*.so*'


/usr/local/lib/python3.10/dist-packages/nvidia/cudnn/lib/libcudnn_engines_runtime_compiled.so.9
/usr/local/lib/python3.10/dist-packages/nvidia/cudnn/lib/libcudnn.so.9
/usr/local/lib/python3.10/dist-packages/nvidia/cudnn/lib/libcudnn_ops.so.9
/usr/local/lib/python3.10/dist-packages/nvidia/cudnn/lib/libcudnn_cnn.so.9
/usr/local/lib/python3.10/dist-packages/nvidia/cudnn/lib/libcudnn_adv.so.9
/usr/local/lib/python3.10/dist-packages/nvidia/cudnn/lib/libcudnn_graph.so.9
/usr/local/lib/python3.10/dist-packages/nvidia/cudnn/lib/libcudnn_heuristic.so.9
/usr/local/lib/python3.10/dist-packages/nvidia/cudnn/lib/libcudnn_engines_precompiled.so.9


## Pytorch way

In [20]:
print(torch.backends.cudnn.enabled)
print(torch.backends.cudnn.version())

True
90100


## Troubleshooting → brute force pythonic way

In [35]:
import os
import re

def find_cudnn_header(start_path='/usr', filename='cudnn.h'):
    for root, _, files in os.walk(start_path):
        if 'cudnn.h' in files:
            cudnn_header_path = os.path.join(root, filename)
            print(f"Found cudnn.h at: {cudnn_header_path}")
            return cudnn_header_path
    return None

def extract_cudnn_version(filepath):
    with open(filepath, 'r') as f:
        content = f.read()    
    major = re.search(r'#define\s+CUDNN_MAJOR\s+(\d+)', content)
    minor = re.search(r'#define\s+CUDNN_MINOR\s+(\d+)', content)
    patch = re.search(r'#define\s+CUDNN_PATCHLEVEL\s+(\d+)', content)
    return major, minor, patch

In [36]:
cudnn_header = None
cudnn_header = find_cudnn_header()

if cudnn_header:
    major, minor, patch = extract_cudnn_version(cudnn_header)
    
    if not major and not minor and not patch:
        cudnn_header = find_cudnn_header(filename="cudnn_version.h")
        if cudnn_header:
            major, minor, patch = extract_cudnn_version(cudnn_header)
    
    print(f"#define CUDNN_MAJOR {major.group(1)}" if major else "CUDNN_MAJOR not found")
    print(f"#define CUDNN_MINOR {minor.group(1)}" if minor else "CUDNN_MINOR not found")
    print(f"#define CUDNN_PATCHLEVEL {patch.group(1)}" if patch else "CUDNN_PATCHLEVEL not found")
else:
    print("cudnn.h not found")

Found cudnn.h at: /usr/local/lib/python3.10/dist-packages/nvidia/cudnn/include/cudnn.h
Found cudnn.h at: /usr/local/lib/python3.10/dist-packages/nvidia/cudnn/include/cudnn_version.h
#define CUDNN_MAJOR 9
#define CUDNN_MINOR 1
#define CUDNN_PATCHLEVEL 0
